# Data Preprocessing
* change to space splitted basis.
* remove duplicate rows and rows that consist too few element.

# Train Gensim Models - Word2Vec

In [34]:
from gensim import corpora
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
from pprint import pprint
import gensim
import gensim.downloader as api
import numpy as np
import pandas as pd

In [ ]:
# train word2vec model by genism, with data: text8
dataset = api.load('text8')
data = [d for d in dataset]

# train word2vec model. Default result vector size = 100
model = Word2Vec(data, min_count=0, workers=cpu_count())

model.save('./gensim_models/word2vec_model_text8')

In [ ]:
# update our model with tweet dataset

# create copora dictionary
tweet_data = pd.read_csv('./datasets/tweet_stopwords_duplicates_short_removed_space_splitted.csv')
data_list = tweet_data.iloc[:, 0].tolist()
# tokenize data into words
texts = [[text for text in tweet.split()] for tweet in data_list]
# create dictionary by copora
dictionary = corpora.Dictionary(texts)
print(type(dictionary))
print(type(texts))


# load text8 word2vec dataset
model = Word2Vec.load('./gensim_models/word2vec_model_text8')
print(model['media'], '\n')

# update the model with tweets dataset
model.build_vocab(texts, update=True)
model.train(data, total_examples=model.corpus_count, epochs=model.iter)
print(model['media'])

# save the model
model.save('./gensim_models/word2vec_model_text8_tweet')

In [ ]:
# train a word2vec model with tweet dataset only

# create copora dictionary
tweet_data = pd.read_csv('./datasets/tweet_stopwords_duplicates_short_removed_space_splitted.csv')
data_list = tweet_data.iloc[:, 0].tolist()
# tokenize data into words
texts = [[text for text in tweet.split()] for tweet in data_list]
# create dictionary by copora
dictionary = corpora.Dictionary(texts)
print(type(dictionary))
print(type(texts))

# train word2vec model. Default result vector size = 100
model = Word2Vec(texts, min_count=0, workers=cpu_count())
print(model['trump'], '\n')
print(model['taiwan'])

model.save('./gensim_models/word2vec_model_tweet')

In [ ]:
# check the result with only tweets data
print(model.wv.most_similar('trump'), '\n')
print(model.wv.most_similar('taiwan'), '\n')
print(model.wv.most_similar('obama'), '\n')
print(model.wv.most_similar('freak'), '\n')

# Gensim Training - Doc2Vec

In [45]:
import gensim
import gensim.downloader as api
import numpy as np
import pandas as pd

In [ ]:
# function to create 'tagged document'
def create_tagged_document(list_of_list_of_word):
    for i, list_of_word in enumerate(list_of_list_of_word):
        yield gensim.models.doc2vec.TaggedDocument(list_of_word, [i])

In [ ]:
# create 'tagged document' dataset of text8
# load text8 dataset
dataset = api.load('text8')
data = [d for d in dataset]

# create tagged document of text8
training_data_text8 = list(create_tagged_document(data))

In [ ]:
# initialize doc2vec model
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=100)

# build the vocabulary
model.build_vocab(training_data_text8)

# train the Doc2Vec model
model.train(training_data_text8, total_examples=model.corpus_count, epochs=model.epochs)

model.save('./gensim_models/doc2vec_model_text8')

In [ ]:
# create a dataset with text8 and tweets data
tweet_data = pd.read_csv('./datasets/tweet_stopwords_duplicates_short_removed_space_splitted.csv')
data_list = tweet_data.iloc[:, 0].tolist()

# tokenize data into words
training_tweet = [[text for text in tweet.split()] for tweet in data_list]

# create tagged document with text8 and tweet
training_data_text8_tweet = list(create_tagged_document(data + training_tweet))

# create tagged document with tweet only
training_data_tweet = list(create_tagged_document(training_tweet))

In [ ]:
# create a model for both text8 and tweets data
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=100)

# build the vocabulary
model.build_vocab(training_data_text8_tweet)

# train the model
model.train(training_data_text8_tweet, total_examples=model.corpus_count, epochs=model.epochs)

model.save('./gensim_models/doc2vec_model_text8_tweet')

# Create Tagged dataset and gensim models

In [46]:
import gensim
import gensim.downloader as api
import numpy as np
import pandas as pd

In [47]:
# function to create 'tagged document'
def create_tagged_document(list_of_list_of_word):
    for i, list_of_word in enumerate(list_of_list_of_word):
        yield gensim.models.doc2vec.TaggedDocument(list_of_word, [i])

In [3]:
# prepare data by concating original and removed stopwords together
original = pd.read_csv("./datasets/12_original_text_only_cleaned.csv", sep='\t')
retweet = pd.read_csv("./datasets/11_retweet_text_only_cleaned.csv", sep='\t')

print(len(original))
print(original.columns)
print(len(retweet))
print(retweet.columns)

30567
Index(['is_RT', 'retweet_from', 'whole_tweet_text', 'original_text',
       'retweet_text', 'cleaned_original_text'],
      dtype='object')
614
Index(['is_RT', 'retweet_from', 'whole_tweet_text', 'original_text',
       'retweet_text', 'cleaned_retweet_text'],
      dtype='object')


In [50]:
# tokenize data into words
training_tweet = [[text for text in str(tweet).split()] for tweet in retweet.loc[:, 'cleaned_retweet_text']]
# training_tweet = [[text for text in str(tweet).split()] for tweet in original.loc[:, 'cleaned_original_text']]

# create tagged document with tweet only
training_data_tweet = list(create_tagged_document(training_tweet))

# create a model for only tweets
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=100)

# build the vocabulary
model.build_vocab(training_data_tweet)

# train the model
model.train(training_data_tweet, total_examples=model.corpus_count, epochs=model.epochs)

model.save('./gensim_models/doc2vec_model_retweet_new')
# model.save('./gensim_models/doc2vec_model_original_new')

# Create dataset with Doc2Vec Features

In [4]:
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm

In [5]:
model = gensim.models.doc2vec.Doc2Vec.load('./gensim_models/doc2vec_model_retweet_new')

for i, j in enumerate(tqdm(retweet.loc[:, 'cleaned_retweet_text'])):
    element_splitted = str(j).split(" ")
    feature_array = model.infer_vector(element_splitted)
    for a, b in enumerate(feature_array):
        column_name = 'feature_' + str(a)
        retweet.loc[i, column_name] = b

print(retweet.head())
print(retweet.describe())
retweet.to_csv('./datasets/13_retweet_text_doc2vec_new.csv', sep='\t', header=True, index=None)

100%|██████████| 614/614 [00:12<00:00, 48.13it/s]


   is_RT     retweet_from                                   whole_tweet_text  \
0      1    SteveSGoddard  RT @SteveSGoddard: The @nytimes was thrilled b...   
1      1   DonaldJTrumpJr  RT @DonaldJTrumpJr: Happy new year everyone. #...   
2      1  VoteHillary2016  RT @VoteHillary2016: Trump says Taiwain leader...   
3      1   NeonKnight1337  RT @NeonKnight1337: Donald Trump's call with T...   
4      1    gatewaypundit  RT @gatewaypundit: The Trump Hotel Waikiki loo...   

  original_text                                       retweet_text  \
0           NaN   The @nytimes was thrilled by Obama talking to...   
1           NaN   Happy new year everyone. #newyear #family #va...   
2           NaN   Trump says Taiwain leader called him, though ...   
3           NaN   Donald Trump's call with Taiwan president was...   
4           NaN   The Trump Hotel Waikiki looks like a lovely r...   

                                cleaned_retweet_text  feature_0  feature_1  \
0  the nytimes was t

In [6]:
model = gensim.models.doc2vec.Doc2Vec.load('./gensim_models/doc2vec_model_original_new')

for i, j in enumerate(tqdm(original.loc[:, 'cleaned_original_text'])):
    element_splitted = str(j).split(" ")
    feature_array = model.infer_vector(element_splitted)
    for a, b in enumerate(feature_array):
        column_name = 'feature_' + str(a)
        original.loc[i, column_name] = b

print(original.head())
print(original.describe())
original.to_csv('./datasets/14_original_text_doc2vec_new.csv', sep='\t', header=True, index=None)

100%|██████████| 30567/30567 [16:37<00:00, 30.65it/s]


   is_RT  retweet_from                                   whole_tweet_text  \
0      0           NaN  "@MysticWolf12001: @realDonaldTrump C'mon, you...   
1      0           NaN  "@Chad_Williams91: @realDonaldTrump if you're ...   
2      0           NaN  "@HunterBalthazor: @realDonaldTrump if you ran...   
3      0           NaN  "@kyleraccio: @realDonaldTrump @Vinny_Titone I...   
4      0           NaN                @HarryCraig96 @TrumpTowerNY Thanks!   

                                       original_text  retweet_text  \
0  "@MysticWolf12001: @realDonaldTrump C'mon, you...           NaN   
1  "@Chad_Williams91: @realDonaldTrump if you're ...           NaN   
2  "@HunterBalthazor: @realDonaldTrump if you ran...           NaN   
3  "@kyleraccio: @realDonaldTrump @Vinny_Titone I...           NaN   
4                @HarryCraig96 @TrumpTowerNY Thanks!           NaN   

                               cleaned_original_text  feature_0  feature_1  \
0  mysticwolf12001 cmon you know youre

# Gensim Vector Relations Check

In [12]:
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm

In [13]:
# checking words similarities

model = gensim.models.doc2vec.Doc2Vec.load('./gensim_models/doc2vec_model_retweet_new')

print('happy\n', model.wv.most_similar('happy'), '\n')
print('nytimes\n', model.wv.most_similar('nytimes'), '\n')
print('trump\n', model.wv.most_similar('trump'), '\n')
print('realdonaldtrump\n', model.wv.most_similar('trump'), '\n')
print('obama\n', model.wv.most_similar('obama'), '\n')
print('taiwan\n', model.wv.most_similar('taiwan'), '\n')
print('china\n', model.wv.most_similar('china'), '\n')
print('beijing\n', model.wv.most_similar('beijing'), '\n')
print('cnn\n', model.wv.most_similar('cnn'), '\n')

happy
 [('birthday', 0.9392526149749756), ('hampshire', 0.9201045036315918), ('x1f1fax1f1f8', 0.8853327035903931), ('donaldjtrumpjr', 0.8516042232513428), ('simply', 0.8400300741195679), ('thank', 0.8388058543205261), ('peace', 0.8381062150001526), ('nevada', 0.8368520736694336), ('primary', 0.8350822925567627), ('incredible', 0.832456111907959)] 

nytimes
 [('dnc', 0.9539414644241333), ('foolish', 0.9511896371841431), ('taiwanese', 0.9431979656219482), ('interest', 0.9420510530471802), ('blasts', 0.9390931725502014), ('conflict', 0.9382774829864502), ('means', 0.9286386370658875), ('former', 0.9256524443626404), ('hysterical', 0.9247620105743408), ('arranged', 0.9226740598678589)] 

trump
 [('taiwan', 0.8802579045295715), ('arranged', 0.8785498738288879), ('taiwanese', 0.8737508058547974), ('spoke', 0.8711260557174683), ('msm', 0.8688294887542725), ('telephone', 0.8656235337257385), ('foolish', 0.8620258569717407), ('development', 0.8605897426605225), ('which', 0.8572181463241577), ('

In [14]:
# print out vector of words in our dataset

print(model['trump'], '\n')
print(model['taiwan'], '\n')

[-0.20089632 -0.43371838  0.40903682  0.5232539  -0.09847506  0.06498223
 -0.5256974   0.06251621 -0.67933595  0.61427474 -0.17155817  0.0666979
 -0.13634105  0.08373062 -0.42614767 -0.04508951 -0.51630783 -0.78576875
 -0.06757877  0.21523958  0.40046397  0.26988435 -0.2397853   0.38511437
  0.01208746 -0.7255458  -0.6818082   0.4965771  -1.0080531  -0.9309631
  0.08889045 -0.49097294  0.28477725  0.43024912 -0.70103633  0.41717553
 -0.54872406 -0.34307778 -0.951142    0.1078896   0.02034028 -0.5034523
 -0.68035924 -0.7232569  -0.545198    0.6891764  -0.54035807  0.32911068
  0.23385102 -0.7544698 ] 

[-0.04689683 -0.6998818   0.96006435 -0.06521887 -0.19370526 -0.09437462
 -0.84830284 -0.32293    -1.3422617   0.5378286  -0.6157186   0.1898993
  0.01309427  0.19617535 -0.31892866 -1.0963701  -1.3945589  -1.3643827
  0.3050841   0.32039866  0.34118     0.58407784 -0.3048922   0.44602215
  0.52530587 -1.2686615  -0.70920163  0.3370944  -1.2055398  -1.2726709
  0.24273778 -0.42144412  0.5

In [18]:
# reference: https://stackoverflow.com/questions/42781292/doc2vec-get-most-similar-documents
word_tokens = "the nytimes was thrilled by obama talking to communist dictator castro and horrified by trump talking to the elected le".split(" ")
token_vector = model.infer_vector(word_tokens)
sims = model.docvecs.most_similar([token_vector])
print(sims)

# print out the original text
data = pd.read_csv("./datasets/13_retweet_text_doc2vec_new.csv", sep='\t')
print("the nytimes was thrilled by obama talking to communist dictator castro and horrified by trump talking to the elected le")
print('\n', data.loc[183, 'retweet_text'], '\n')
print(data.loc[584, 'retweet_text'], '\n')
print(data.loc[118, 'retweet_text'], '\n')
print(data.loc[455, 'retweet_text'], '\n\n')


word_tokens = "incompetence is really the primary threat from the trump administration".split(" ")
token_vector = model.infer_vector(word_tokens)
sims = model.docvecs.most_similar([token_vector])
print(sims)

# print out the original text
data = pd.read_csv("./datasets/13_retweet_text_doc2vec_new.csv", sep='\t')
print('\n', data.loc[38, 'retweet_text'], '\n')
print(data.loc[569, 'retweet_text'], '\n')
print(data.loc[40, 'retweet_text'], '\n')
print(data.loc[324, 'retweet_text'], '\n\n')


word_tokens = "obama turned his back on taiwan".split(" ")
token_vector = model.infer_vector(word_tokens)
sims = model.docvecs.most_similar([token_vector])
print(sims)

# print out the original text
data = pd.read_csv("./datasets/13_retweet_text_doc2vec_new.csv", sep='\t')
print('\n', data.loc[128, 'retweet_text'], '\n')
print(data.loc[579, 'retweet_text'], '\n')
print(data.loc[134, 'retweet_text'], '\n')
print(data.loc[162, 'retweet_text'], '\n\n')

[(0, 0.8908886909484863), (183, 0.7266837954521179), (584, 0.6549695134162903), (118, 0.6189082264900208), (455, 0.6054126620292664), (398, 0.5983391404151917), (31, 0.5962131023406982), (530, 0.5778654217720032), (166, 0.5736516714096069), (113, 0.5693560838699341)]
the nytimes was thrilled by obama talking to communist dictator castro and horrified by trump talking to the elected le

  ONLY REASON it was "protocol" to not talk to Taiwan publicly is Obama is too cowardly &amp; TERRIFIED of offending China.??? 

 Corrupt MSM says President-Elect Trump is not allowed to speak w/ Taiwan because Communist China says so. China does NOT tell??n 

 Donald Trump fell for a ?ittle trick??performed by Taiwan, China says https://t.co/64eU35xFlIcede 

 Story is not: Trump broke diplomatic norms by calling Taiwan. 


[(38, 0.8712615966796875), (40, 0.6251914501190186), (569, 0.6199179887771606), (324, 0.5989843606948853), (120, 0.5843089818954468), (316, 0.5767746567726135), (393, 0.5718825459480

In [32]:
data = pd.read_csv('./datasets/10_original_text_doc2vec.csv', sep='\t')